# EEG Explorer

## 1. Trabalhando com dados brutos

In [8]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import scipy
import os

from src.file import File
from src.data import Formatter

### 1.1. Importando arquivos do bucket

In [11]:
raw = File.get_files_from(resource="raw")

df = pd.read_csv(f"{File.get_path_by(resource='raw')}/{raw[0]}", delimiter="\t")
df.head()

,Unnamed: 0,Index,Fp1,Fp2,C3,C4,P7,P8,O1,O2,...,other.6,other.7,other.8,other.9,other.10,other.11,other.12,Timestamp,other.13,Timestamp (Formatted)
0,0,0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,28.0,48.0,7.0,192.0,0.0,0.0,0.0,1.670415e+09,0.0,2022-12-07 09:16:01.548664
1,1,1.0,-116197.586048,70213.968802,-157452.285324,-150474.161549,-19826.878537,-154831.321625,66806.939425,-66900.016093,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.670415e+09,0.0,2022-12-07 09:16:01.564580
2,2,2.0,-105795.830102,43769.109207,-112905.233282,-122227.513975,-64803.333723,-114433.038929,569.534549,-82343.003642,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.670415e+09,0.0,2022-12-07 09:16:01.575633
3,3,3.0,-114567.880971,28118.067060,-97444.815851,-105830.188207,-64555.828908,-97760.274210,-33991.050097,-90947.959156,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.670415e+09,0.0,2022-12-07 09:16:01.579660
4,4,4.0,-137846.248156,64758.974920,-156445.929614,-143579.808281,-35740.413418,-150625.859354,18197.766831,-96318.682106,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.670415e+09,0.0,2022-12-07 09:16:01.582661


### 1.2. Renomeando arquivos brutos

In [12]:
File.rename_raw_files()

### 1.3. Formatando os sinais

In [13]:
renamed = File.get_files_from(resource="renamed")
dataframes = [pd.read_csv(f"{File.get_path_by(resource='renamed')}/{file}", delimiter="\t") for file in renamed]

#### 1.3.1. Normalizando timestamps

In [14]:
# df["Timestamp"] = df["Timestamp"] - df["Timestamp"].min()

dataframes = Formatter.normalize_timestamps_for(dataframes)

#### 1.3.2. Removendo colunas indesejadas

In [15]:
# df.drop de cada coluna marcada como "other", "Unnamed" ou "Timestamp (Formatted)"

dataframes = Formatter.remove_other_columns_for(dataframes)

##### DataFrame pós formatação

In [16]:
dataframes[0].head()

,Index,Fp1,Fp2,C3,C4,P7,P8,O1,O2,Timestamp
0,0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000
1,1.0,-30684.333551,-54767.279418,164410.104314,-35547.368942,30972.441997,92641.290661,-46111.364923,-74367.728236,0.017010
2,2.0,-22767.318726,-38787.579878,151049.332509,-109237.703277,62394.899205,98603.120244,-447.936799,-31001.693575,0.020050
3,3.0,4058.659344,-7946.013541,183130.925096,-142526.406932,-111357.091297,-54117.613231,-164388.954829,-177441.745461,0.024025
4,4.0,1175.880705,-17243.740964,200981.666048,-103547.672195,-123177.869331,-75500.693269,-161731.177000,-173546.725964,0.029008


In [17]:
File.write_dataframes_in(path=File.get_path_by(resource="formatted"),
                         dataframes=dataframes,
                         filenames=renamed)

### 1.4. Visualização dos sinais

In [17]:
# Dependências
filenames = File.get_files_from(resource="formatted")
channels = ["Fp1", "Fp2", "C3", "C4", "P7", "P8", "O1", "O2"]

# Widgets
window_widget = widgets.IntSlider(min=1,
                                  max=1000,
                                  step=10,
                                  value=50,
                                  description="Janela")
file_widget = widgets.Combobox(options=filenames, 
                               value=filenames[0],
                               description="Arquivo")
channel_widget = widgets.Dropdown(options=channels,
                                  value=channels[0],
                                  description="Canal")

# Função de plotagem
def plot_signal(file, channel, window):
    delta = window
    df = pd.read_csv(f"{File.get_path_by(resource='formatted')}/{file}", delimiter=",")
    timestamps = df["Timestamp"].to_numpy()
    channel_data = df[channel].to_numpy()
    
    moving_average = np.convolve(channel_data, np.ones(512)/512, mode="valid")
    first_derivative = 100*(moving_average[:-delta] - moving_average[delta:])/delta
    fig = px.line(x=timestamps[:len(first_derivative)],
                  y=[channel_data[:len(first_derivative)],
                     moving_average[:len(first_derivative)],
                     first_derivative])
    fig.update_yaxes(range=[moving_average[10]-5E3, moving_average[10]+5E3])
    fig.show()

# Display dos widgets
out = widgets.interactive_output(plot_signal, 
                                 {"file": file_widget, 
                                  "channel": channel_widget, 
                                  "window": window_widget})

widgets.VBox([widgets.HBox([file_widget, channel_widget, window_widget]), out])

## 2. Filtrando trechos indesejados dos sinais

In [1]:
import ipywidgets as widgets
import os

from src.file import File
from src.data import Truncate
from src.data import TruncateIntervals
from src.data import Plotter

### 2.1. Seleção de intervalos com os trechos indesejados para cada sinal

#### 2.1.1. Selecionando trechos de maneira interativa

In [10]:
# Dependências
output = widgets.Output()
trunc_intervals = TruncateIntervals(truncate_intervals_path=File.get_path_by(resource="truncation_intervals"))
plotter = Plotter(files_path=File.get_path_by(resource="formatted"),
                  output=output)
filenames = File.get_files_from(resource="formatted")
channels = ["Fp1", "Fp2", "C3", "C4", "P7", "P8", "O1", "O2"]

file_widget_changed = False
channel_widget_changed = False

# Widgets
file_w = widgets.Combobox(options=filenames,
                          value=filenames[0],
                          description="Arquivo",
                          layout=widgets.Layout(width="300px"))
channel_w = widgets.Dropdown(options=channels,
                             value = channels[0],
                             description="Canal",
                             layout=widgets.Layout(width = "300px"))
refresh_btn_w = widgets.Button(description="Refresh",
                               layout=widgets.Layout(width = "70px"))
refresh_btn_w.layout.margin = "0px 0px 0px 50px"

signal_controller_hboxw = widgets.HBox([file_w, channel_w, refresh_btn_w])

label_w = widgets.Label(value="Intervalo de corte")
bottom_limit_w = widgets.FloatText(layout=widgets.Layout(width="100px"))
top_limit_w = widgets.FloatText(layout = widgets.Layout(width="100px"))
add_interval_btn_w = widgets.Button(description="Add",
                                    layout=widgets.Layout(width="70px"))
pop_interval_btn_w = widgets.Button(description="Pop",
                                    layout=widgets.Layout(width="70px"))
save_interval_btn_w = widgets.Button(description="Save",
                                     layout=widgets.Layout(width="70px"))

trunc_intervals_hboxw = widgets.HBox([label_w,
                                      bottom_limit_w,
                                      top_limit_w,
                                      add_interval_btn_w,
                                      pop_interval_btn_w,
                                      save_interval_btn_w],
                                     layout=widgets.Layout(justify_content='center'))

# Definição de eventos para os widgets
def file_widget_change_handler(change):
    global file_widget_changed
    if change["type"] == "change" and change["name"] == "value":
        file_widget_changed = True

def channel_widget_change_handler(change):
    global channel_widget_changed
    if change["type"] == "change" and change["name"] == "value":
        channel_widget_changed = True

def refresh_button_clicked(btn):
    global file_widget_changed
    global channel_widget_changed
    if file_widget_changed:
        trunc_intervals.save_current_file_intervals()
        trunc_intervals.load_file_intervals(file_w.value)
        plotter.load_signal(file_w.value, channel_w.value)
        plotter.plot_signal(trunc_intervals.get_channel_intervals(channel_w.value))
        file_widget_changed = False
        channel_widget_changed = False
    elif channel_widget_changed:
        plotter.change_current_fig(channel_w.value)
        plotter.plot_signal(trunc_intervals.get_channel_intervals(channel_w.value))
        channel_widget_changed = False

def add_intervals_clicked(btn):
    start = bottom_limit_w.value
    end = top_limit_w.value
    trunc_intervals.add_interval_by_channel(channel_w.value, start, end)
    plotter.plot_signal(trunc_intervals.get_channel_intervals(channel_w.value))

def pop_intervals_clicked(btn):
    trunc_intervals.pop_interval_by_channel(channel_w.value)
    plotter.plot_signal(trunc_intervals.get_channel_intervals(channel_w.value))

def save_intervals_clicked(btn):
    trunc_intervals.save_current_file_intervals()

file_w.observe(file_widget_change_handler)
channel_w.observe(channel_widget_change_handler)
refresh_btn_w.on_click(refresh_button_clicked)
add_interval_btn_w.on_click(add_intervals_clicked)
pop_interval_btn_w.on_click(pop_intervals_clicked)
save_interval_btn_w.on_click(save_intervals_clicked)

# Display dos widgets
trunc_intervals.load_file_intervals(file_w.value)
plotter.load_signal(file_w.value, channel_w.value)
plotter.plot_signal(trunc_intervals.get_channel_intervals(channel_w.value))

display(signal_controller_hboxw, output, trunc_intervals_hboxw)

Output()

### 2.2. Truncamento dos sinais originais a partir dos intervalos selecionados

In [11]:
intervals_origin_dir = \
    "pre_calculated_truncation_intervals" \
    if input("Deseja utilizar intervalos pré calculados? [y/n] ") == "y" else "truncation_intervals"

Deseja utilizar intervalos pré calculados? [y/n]  y


In [12]:
print(f"'{intervals_origin_dir}' selecionado!")

'pre_calculated_truncation_intervals' selecionado!


In [13]:
truncation_intervals = File.get_files_from(resource=intervals_origin_dir)
filenames = [file.replace('json', 'csv') for file in truncation_intervals]

trc = Truncate(files_path=File.get_path_by(resource="formatted"),
               trunc_intervals_path=File.get_path_by(resource=intervals_origin_dir))

truncated_dfs = []
for file in filenames:
    trc.setup_by_filename(file)
    df = trc.truncate()
    truncated_dfs.append(df)

File.write_dataframes_in(path=File.get_path_by(resource="truncated"),
                         dataframes=truncated_dfs,
                         filenames=filenames)

## 3. Preparando arquivos para a fase de treino da rede neural

In [1]:
from src.file import File
from src.data import Continuous

### 3.1. Separando arquivos originais em N arquivos com sinal contínuo

In [2]:
intervals_origin_dir = \
    "pre_calculated_truncation_intervals" \
    if input("Deseja utilizar intervalos pré calculados? [y/n] ") == "y" else "truncation_intervals"

Deseja utilizar intervalos pré calculados? [y/n]  y


In [3]:
print(f"'{intervals_origin_dir}' selecionado!")

'pre_calculated_truncation_intervals' selecionado!


In [4]:
cnts = Continuous(input_data_path=File.get_path_by(resource="truncated"),
                  output_data_path=File.get_path_by(resource="continuous"),
                                                   truncate_intervals_path=File.get_path_by(resource=intervals_origin_dir))
truncated = File.get_files_from(resource="truncated")

for file in truncated:
    cnts.process_file(file)

### 3.2. Adicionando arquivos não truncados junto aos fragmentos contínuos

In [5]:
File.add_not_fragmented_files_to_continuous()

### 3.3. Separando arquivos em treino e teste

In [6]:
File.generate_train_test_files()